# Análise de Inteligência Comercial - FMCG Doces e Guloseimas
## ETL → EDA → Forecast → Dashboard Export
Neste notebook, realizamos:
1. ETL nas bases **Sell In** e **Sell Out**
2. Análise Exploratória de Dados (EDA)
3. Forecast de vendas (próxima seção)
4. Exportação para dashboard
---

In [1]:
from pathlib import Path

# Definição de diretórios
BASE_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'

# Cria processed se não existir
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

# Mapeamento mês-abreviado → número
mes_map = {
    'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,
    'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8,
    'set': 9, 'out': 10, 'nov': 11, 'dez': 12
}

def parse_mes_ano(x):
    s = str(x).strip().lower()
    if '/' in s:
        try:
            return datetime.strptime(s, '%m/%Y')
        except:
            pass
    parts = s.split()
    if len(parts) == 2:
        mes_abbr, ano = parts
        mes = mes_map.get(mes_abbr[:3])
        if mes:
            return datetime(int(ano), mes, 1)
    raise ValueError(f'Formato de data não reconhecido: {x!r}')

### 1. ETL da Base Sell In

In [3]:
# 1.1 Carrega raw
sell_in = pd.read_excel(RAW_DIR / 'base sell in.xlsx')
sell_in.columns = sell_in.columns.str.strip().str.replace(r'[\r\n\t]+',' ', regex=True)

# 1.2 Parse de datas
sell_in['Data_SellIn'] = sell_in['Calendario[Mês/Ano]'].apply(parse_mes_ano)
sell_in['Ano_Mes'] = sell_in['Data_SellIn'].dt.strftime('%Y-%m')

# 1.3 Converte numéricos
sell_in['[SumP_NF_ValTotal]'] = sell_in['[SumP_NF_ValTotal]'].replace(['-',''], np.nan).astype(float)
sell_in['[SumPeso_Liquido]'] = sell_in['[SumPeso_Liquido]'].replace(['-',''], np.nan).astype(float)
sell_in['[SumP_NF_Quantidade]'] = pd.to_numeric(sell_in['[SumP_NF_Quantidade]'], errors='coerce').fillna(0).astype(int)

# 1.4 Filtra vendas válidas
sell_in = sell_in[(sell_in['FATO[P.NF.OperacaoFini]']=='Venda') &
                  (~sell_in['Produto[P.Produtos.LinhaProducao]'].str.contains('PROMOCIONAIS', na=False, case=False)) &
                  (sell_in['[SumP_NF_ValTotal]'].notna())].copy()

# 1.5 Ajuste Tipo_Produto
mask = (sell_in['Produto[P.Produtos.LinhaProducao]']=='MARSHMALLOW') & sell_in['Produto[P.Produtos.TpPro.Inteligente]'].fillna('').eq('')
sell_in.loc[mask,'Produto[P.Produtos.TpPro.Inteligente]']='MARSH'

# 1.6 Renomeia colunas
sell_in.rename(columns={
    'FATO[P.NF.Ramo_Atividade]':'Ramo_Atividade',
    'Cliente[P.Cliente.Pais]':'Pais','Cliente[P.Cliente.Regiao]':'Regiao',
    'Cliente[P.Cliente.NomeFantasia]':'Cliente','Cliente[P.Cliente.UF]':'UF',
    'Produto[P.Produtos.LinhaProducao]':'Linha_Producao','Produto[P.Produtos.Gramagem]':'Gramagem',
    'Produto[P.Produtos.MacroFormato]':'Macro_Formato','Produto[P.Produtos.Nome]':'Nome_Produto',
    'Produto[P.Produtos.Grupo]':'Grupo_Produto','Produto[P.Produtos.TpPro.Inteligente]':'Tipo_Produto',
    'Cliente[P.Cliente.Municipio]':'Municipio','Hierarquia[P.Cliente.Canal_Hierarquia]':'Canal',
    'Hierarquia[P.Cliente.GerenteRegional]':'Gerente_Regional','Hierarquia[P.Cliente.VendedorNome]':'Vendedor',
    'Produto[Cod. Produto]':'Cod_Produto','[SumP_NF_Quantidade]':'Quantidade',
    '[SumP_NF_ValTotal]':'Valor_Total','[SumPeso_Liquido]':'Peso_Liquido'
}, inplace=True)

print('Sell In tratado:')
print(sell_in.info())

Sell In tratado:
<class 'pandas.core.frame.DataFrame'>
Index: 4172 entries, 0 to 13305
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Calendario[Mês/Ano]      4172 non-null   object        
 1   Ramo_Atividade           4172 non-null   object        
 2   Pais                     4172 non-null   object        
 3   Regiao                   4172 non-null   object        
 4   FATO[P.NF.OperacaoFini]  4172 non-null   object        
 5   Cliente                  4172 non-null   object        
 6   UF                       4172 non-null   object        
 7   Linha_Producao           4172 non-null   object        
 8   Gramagem                 4172 non-null   object        
 9   Macro_Formato            4172 non-null   object        
 10  Nome_Produto             4172 non-null   object        
 11  Grupo_Produto            4172 non-null   object        
 12  Tipo_Produto         

C:\Users\frdsz\AppData\Local\Temp\ipykernel_26052\4070617124.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sell_in['[SumP_NF_ValTotal]'] = sell_in['[SumP_NF_ValTotal]'].replace(['-',''], np.nan).astype(float)
C:\Users\frdsz\AppData\Local\Temp\ipykernel_26052\4070617124.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sell_in['[SumPeso_Liquido]'] = sell_in['[SumPeso_Liquido]'].replace(['-',''], np.nan).astype(float)


### 2. ETL da Base Sell Out

In [4]:
# 2.1 Carrega raw
sell_out = pd.read_excel(RAW_DIR / 'BASE SELL OUT.xlsx')
sell_out.columns = sell_out.columns.str.strip().str.replace(r'[\r\n\t]+',' ', regex=True)

# 2.2 Datas
sell_out['Data'] = pd.to_datetime(sell_out['Ano/Mês'], errors='raise')
sell_out['Ano_Mes'] = sell_out['Data'].dt.strftime('%Y-%m')

# 2.3 Numéricos
for col in ['Sell through valor','Sell through cx']:
    sell_out[col] = pd.to_numeric(
        sell_out[col].astype(str).str.replace(r'[^0-9,.-]','', regex=True).str.replace(',','.'), errors='coerce'
    )
sell_out['Sell through unidade'] = pd.to_numeric(sell_out['Sell through unidade'], errors='coerce').fillna(0).astype(int)

# 2.4 Filtra positivos
sell_out = sell_out[sell_out['Sell through valor']>0].copy()

# 2.5 Renomeia
sell_out.rename(columns={
    'Agente de Distribuição':'Agente_Distribuicao','UF PDV':'UF_PDV','Cidade PDV':'Cidade_PDV',
    'Segmentação Mtrix':'Segmentacao_Mtrix','Cód Produto (SKU)':'Cod_Produto','Nome Produto':'Nome_Produto',
    'Sell through valor':'Valor_SellThrough','Sell through unidade':'Unidades_SellThrough',
    'Sell through cx':'Caixas_SellThrough'
}, inplace=True)

print('Sell Out tratado:')
print(sell_out.info())

Sell Out tratado:
<class 'pandas.core.frame.DataFrame'>
Index: 25708 entries, 0 to 77275
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Ano/Mês               25708 non-null  datetime64[ns]
 1   Agente_Distribuicao   25708 non-null  object        
 2   UF_PDV                25708 non-null  object        
 3   Cidade_PDV            25708 non-null  object        
 4   Segmentacao_Mtrix     25708 non-null  object        
 5   Cod_Produto           25708 non-null  object        
 6   Nome_Produto          25708 non-null  object        
 7   Valor_SellThrough     25708 non-null  float64       
 8   Unidades_SellThrough  25708 non-null  int32         
 9   Caixas_SellThrough    25708 non-null  float64       
 10  Data                  25708 non-null  datetime64[ns]
 11  Ano_Mes               25708 non-null  object        
dtypes: datetime64[ns](2), float64(2), int32(1), object(7)
memory 

### 3. Salvando Bases Tratadas

In [5]:
sell_in.to_csv(PROCESSED_DIR / 'sell_in_processed.csv', index=False)
sell_out.to_csv(PROCESSED_DIR / 'sell_out_processed.csv', index=False)
print('Bases tratadas salvas em data/processed')

Bases tratadas salvas em data/processed


## 4. Análise Exploratória de Dados (EDA)
KPIs globais, séries temporais e performance por produto/região.

In [6]:
import plotly.express as px

sell_in = pd.read_csv(PROCESSED_DIR / 'sell_in_processed.csv', parse_dates=['Data_SellIn'])
sell_out = pd.read_csv(PROCESSED_DIR / 'sell_out_processed.csv', parse_dates=['Data'])

In [7]:
# 4.1 — KPIs
total_sell_in = sell_in['Valor_Total'].sum()
total_sell_out = sell_out['Valor_SellThrough'].sum()
sell_through_rate = total_sell_out / total_sell_in
ticket_medio = total_sell_out / sell_out['Unidades_SellThrough'].sum()

crescimento_mensal = (
    sell_in.groupby('Ano_Mes')['Valor_Total'].sum()
    .pct_change().mul(100).round(2)
)

from IPython.display import Markdown, display
display(Markdown(f"**Total Sell In:** R$ {total_sell_in:,.2f}"))
display(Markdown(f"**Total Sell Out:** R$ {total_sell_out:,.2f}"))
display(Markdown(f"**Sell-Through Rate:** {sell_through_rate:.2%}"))
display(Markdown(f"**Ticket Médio:** R$ {ticket_medio:,.2f}"))
display(Markdown("**Crescimento Mensal (%)**"))
display(crescimento_mensal.to_frame('Crescimento %'))

**Total Sell In:** R$ 106,518,219.36

**Total Sell Out:** R$ 29,901,537.82

**Sell-Through Rate:** 28.07%

**Ticket Médio:** R$ 3.94

**Crescimento Mensal (%)**

,Crescimento %
Ano_Mes,
2023-01,NaN
2023-02,57.61
2023-03,17.27
2023-04,9.57
2023-05,25.26
2023-06,-6.40
2023-07,-45.09
2023-08,79.93
2023-09,39.02


In [8]:
# 4.2 — Séries Temporais Mensais
df_sn = sell_in.groupby('Ano_Mes')['Valor_Total'].sum().reset_index().rename(columns={'Valor_Total':'SellIn'})
df_so = sell_out.groupby('Ano_Mes')['Valor_SellThrough'].sum().reset_index().rename(columns={'Valor_SellThrough':'SellOut'})
df_ts = df_sn.merge(df_so, on='Ano_Mes')

fig = px.line(df_ts, x='Ano_Mes', y=['SellIn','SellOut'],
              labels={'value':'R$','Ano_Mes':'Mês/Ano','variable':'Métrica'},
              title='Evolução Mensal de Sell In vs Sell Out')
fig.update_xaxes(tickangle=45)
fig.show()

In [11]:
# 4.3 — Top 10 Produtos por Sell In
top_prod = sell_in.groupby('Tipo_Produto')['Valor_Total'].sum().nlargest(10).reset_index()
fig = px.bar(top_prod, x='Valor_Total', y='Tipo_Produto', orientation='h',
             title='Top 10 Tipos de Produtos por Sell In (R$)')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [10]:
# 4.4 — Participação por Região
by_region = sell_in.groupby('Regiao')['Valor_Total'].sum().reset_index()
fig = px.pie(by_region, names='Regiao', values='Valor_Total',
             title='Participação de cada Região no Sell In')
fig.show()